In [ ]:
%pip install scikit-learn pandas altair

In [ ]:
# Load iris flower dataset
from sklearn import svm, datasets
iris = datasets.load_iris()

In [ ]:
import pandas as pd
df = pd.DataFrame(iris.data,columns=iris.feature_names)
df['flower'] = iris.target
df['flower'] = df['flower'].apply(lambda x: iris.target_names[x])
df

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.3)

In [ ]:
from sklearn.model_selection import cross_val_score
# Build a Support Vector Classification (SVC) model
model = svm.SVC(kernel='rbf',C=30,gamma='auto')
model.fit(X_train,y_train)
model.score(X_test, y_test)

In [ ]:
# Using a for-loop to iterate over different kernel types and regularization parameter (C) values
import numpy as np
kernels = ['rbf', 'linear']
C = [1,10,20]
avg_scores = {}
for kval in kernels:
    for cval in C:
        cv_scores = cross_val_score(svm.SVC(kernel=kval,C=cval,gamma='auto'),iris.data, iris.target, cv=5)
        avg_scores[kval + '_' + str(cval)] = np.average(cv_scores)

avg_scores

In [ ]:
#Hyperparameter Search with Grid Search

from sklearn.model_selection import GridSearchCV
gs = GridSearchCV(svm.SVC(gamma='auto'), {
    'C': [1,5,10,20,30,50],
    'kernel': ['rbf','linear','poly']
}, cv=5, return_train_score=False)
gs.fit(iris.data, iris.target)

In [ ]:
df = pd.DataFrame(gs.cv_results_)
df

In [ ]:
results = df[['param_C','param_kernel','mean_test_score']]
results

In [ ]:
import altair as alt
alt.Chart(results).mark_rect().encode(
    x='param_C:O',
    y='param_kernel:O',
    color='mean_test_score:Q'
)

In [ ]:
print(f"From GridSearch, this is the best parameter combination {gs.best_params_} and resulting score {gs.best_score_:.2f}.")

In [ ]:
# Hyperparameter Search with Randomized Search

from sklearn.model_selection import RandomizedSearchCV
rs = RandomizedSearchCV(svm.SVC(gamma='auto'), {
        'C': [1,5,10,20,30,50],
        'kernel': ['rbf','linear','poly']
    },
    cv=5,
    return_train_score=False,
    n_iter=2
)
import time
start = time.time()
rs.fit(iris.data, iris.target)
end = time.time()
print(f"RandomizedSearchCV took {end-start:.2f} seconds")
df = pd.DataFrame(rs.cv_results_)[['param_C','param_kernel','mean_test_score']]
df

In [ ]:
start = time.time()
gs.fit(iris.data, iris.target)
end = time.time()
print(f"GridSearchCV took {end-start:.2f} seconds")

In [ ]:
# Model Comparison with Grid Search

from sklearn import svm
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

model_params = {
    'svm': {
        'model': svm.SVC(gamma='auto'),
        'params' : {
            'C': [1,5,10,20,30,50],
            'kernel': ['rbf','linear','poly']
        }
    },
    'random_forest': {
        'model': RandomForestClassifier(),
        'params' : {
            'n_estimators': [1,5,10]
        }
    },
    'logistic_regression' : {
        'model': LogisticRegression(solver='liblinear',multi_class='auto'),
        'params': {
            'C': [1,5,10]
        }
    }
}


In [ ]:
scores = []

for model_name, mp in model_params.items():
    clf =  GridSearchCV(mp['model'], mp['params'], cv=5, return_train_score=False)
    clf.fit(iris.data, iris.target)
    scores.append({
        'model': model_name,
        'best_score': clf.best_score_,
        'best_params': clf.best_params_
    })

df = pd.DataFrame(scores,columns=['model','best_score','best_params'])
df